In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import openpyxl
from datetime import datetime
from helper_functions.db_connections import inst
from email.mime.multipart import MIMEMultipart
import os.path
import pandas as pd
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import datetime as dt
from email.mime.text import MIMEText
import base64
import requests

NEW ---------


In [2]:
# You need coin recon sheet plus the APY/COFA sheet for live waterfall
# I download the sheets and save them as excel files so I have a copy of the raw data (for debugging and archiving)

In [3]:
# Instilend Active Loan book query
query = '''
select
l.loan_number,
b.id as borrower_id,
b.legal_name as borrower_name,
al.symbol as loan_asset_short,
le.quantity as loan_amount,
le.interest_rate,
lc.collateral_asset_short,
lc.collateral_amount
from
instilend.loans l left join instilend.loanelements le on le.loanid_fk = l.id
left join instilend.assets al on le.assetid_fk = al.id
left join instilend.borrowers b on l.borrowerid_fk = b.id
left join instilend.borrower_categories bc on b.categoryid_fk = bc.id
left join instilend.loan_statustypes ls on l.statusid_fk = ls.id
left join (select lc.loanid_fk,
       al.symbol as collateral_asset_short,
       lc.quantity as collateral_amount
FROM instilend.collaterals lc
LEFT JOIN instilend.assets al on lc.assetid_fk = al.id) lc on lc.loanid_fk = l.id
where
ls.description in ('Active')
and l.open_tag=False;
'''
loan_book = inst.create_pandas_table(query)

In [4]:
# Get Borrower Risk info
# Connect to Drive API
spreadsheet_id = '1W1w6hA9hrX3s3DaXlLhFE5tPZd1St8vsMninNgNYtXU'

SCOPES = ['https://www.googleapis.com/auth/drive.readonly']
creds = None
creds = Credentials.from_authorized_user_file('token.json', SCOPES)
service = build('sheets', 'v4', credentials=creds)

# Call the Drive v3 API
SAMPLE_RANGE_NAME = 'Risk Profiles!A1:J92'
request = service.spreadsheets().values().batchGet(
    spreadsheetId=spreadsheet_id,
    ranges=SAMPLE_RANGE_NAME,
    valueRenderOption='FORMATTED_VALUE')
response = request.execute()
df = pd.DataFrame(response['valueRanges'][0]['values']).dropna().reset_index(drop=True)
df.columns = df.loc[0]
df = df.loc[1:]
loan_book_risk = loan_book.merge(df, left_on='borrower_name', right_on='Company Name', how='left')

In [5]:
# Get Funding Rate CnC info
spreadsheet_id = "1K-n3cfBKj0YWxPd_hImkBjZgkL2egrQqKOs1qUcM7rw"
range_ = "FTX Funding Rates!A2:I"
request = service.spreadsheets().values().batchGet(
    spreadsheetId=spreadsheet_id,
    ranges=range_,
    valueRenderOption='UNFORMATTED_VALUE')
response = request.execute()
cnc_apy = pd.DataFrame(response['valueRanges'][0]['values']).dropna().reset_index(drop=True)
cnc_apy.columns = cnc_apy.loc[0]
cnc_apy = cnc_apy.loc[1:, ['Market','24hr']]
cnc_apy.columns = ['Coin','24hr_rate']

In [6]:
interest_rate = pd.DataFrame()
for coin in loan_book['loan_asset_short'].unique().tolist():
    loan_book2 = loan_book[loan_book['loan_asset_short']==coin]
    loan_book2['loan_amount_%'] = loan_book2['loan_amount']/loan_book2['loan_amount'].sum()
    loan_book2['interest_rate_weighted'] = loan_book2['interest_rate'] * loan_book2['loan_amount_%']
    temp = loan_book2.groupby('loan_asset_short')['interest_rate_weighted'].sum().reset_index()
    interest_rate = pd.concat([temp, interest_rate])

/var/folders/q7/qhj4q23x49vb9f__hlp8t3280000gn/T/ipykernel_53933/2019773264.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loan_book2['loan_amount_%'] = loan_book2['loan_amount']/loan_book2['loan_amount'].sum()
/var/folders/q7/qhj4q23x49vb9f__hlp8t3280000gn/T/ipykernel_53933/2019773264.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loan_book2['interest_rate_weighted'] = loan_book2['interest_rate'] * loan_book2['loan_amount_%']


In [7]:
def melt(df, col_vals, key, value):
    # melt pandas dataframe
    # col_vars: list of the columns that will be melted
    # key: name of column needs to be generated
    # value: name of the column that contains the value of interest
    keep_vars = df.columns.difference(col_vals)
    melted = []
    for c in col_vals:
        melted_c = df[keep_vars].copy()
        melted_c[key] = c
        melted_c[value] = df[c]
        melted.append(melted_c)
    return pd.concat(melted)

In [8]:
# CHECK IF THE FREEZE IS UPDATED
waterfall_freeze = pd.read_excel("excel_input/freeze_address.xlsx", sheet_name='Past Waterfall Freezes')
waterfall_freeze.columns = waterfall_freeze.iloc[0]
waterfall_freeze.drop([0], inplace = True)
wfreeze_last_date = str(waterfall_freeze.iloc[-1]["Date"])

freeze = pd.read_excel("excel_input/freeze_address.xlsx")
freeze.columns = freeze.iloc[0]
freeze.drop([0], inplace = True)
freeze_last_date = str(freeze.iloc[-1]["Date"])
freeze_name_date = freeze.iloc[-1]["Date"].strftime('%b %d, %Y')
freeze_date = freeze_last_date.split()[0]
isFreeze = wfreeze_last_date != freeze_last_date

In [9]:
# read the raw spreadsheet, downloanded from shared drive
if isFreeze:
    c_recon = pd.ExcelFile("excel_input/freeze.xlsx")
else:
    c_recon = pd.ExcelFile("excel_input/Celsius_Reconciliation_Master.xlsx")
c_recon = pd.ExcelFile("excel_input/Celsius_Reconciliation_Master.xlsx")
#c_recon = pd.ExcelFile("excel_input/freeze.xlsx")
stats = pd.read_excel(c_recon, "Coin Stats", header = None)
defi = pd.read_excel(c_recon, "DeFi Assets")
ftx = pd.read_excel(c_recon, "FTX Summary")
ftx_cnc = pd.read_excel(c_recon, "FTX Hedge Summary")
defi_main = pd.read_excel(c_recon, "DeFi Main")
template = pd.read_excel("excel_input/coin_apy_template.xlsx", 
                         sheet_name = "APY",
                         header = None)
tier_override = pd.read_excel("excel_input/coin_apy_template.xlsx",
                         sheet_name = "Override_tier_mapping",
                         header = None)
category_override = pd.read_excel("excel_input/coin_apy_template.xlsx",
                         sheet_name = "Override_Category_mapping",
                         header = None)
carry = pd.read_excel("excel_input/coin_apy_template.xlsx", sheet_name='Carry')
manual_adjustements = pd.read_excel("excel_input/coin_apy_template.xlsx", 
                         sheet_name = "Manual_Adjustments")

deployable = pd.read_excel("excel_input/deployable_metric.xlsx",
                           sheet_name="Undeployed & Underdeployed")
template.columns = template.iloc[0]
template = template[template["Coin"].notnull()]
template

,Coin,Bank - Balances,Celsius Network Limited (UK),Celsius Network LLC (US),Celsius Network EU UAB (LT),Celsius Network Finance,Celsius OTC,CEL Treasury,CEL Users,Loans Out,...,CEL TBD,castody,Test - Terra,Test - Terra 2,RD - MARS - TerraUST,DD - Astroport - stLuna,DD - Astroport - stLuna,DD - Curve Polygon - EURTUSD,YD - Lido - stETH,dd-cx-Quickswap-cxBTC/wBTC
0,Coin,Bank - Balances,Celsius Network Limited (UK),Celsius Network LLC (US),Celsius Network EU UAB (LT),Celsius Network Finance,Celsius OTC,CEL Treasury,CEL Users,Loans Out,...,CEL TBD,castody,Test - Terra,Test - Terra 2,RD - MARS - TerraUST,DD - Astroport - stLuna,DD - Astroport - stLuna,DD - Curve Polygon - EURTUSD,YD - Lido - stETH,dd-cx-Quickswap-cxBTC/wBTC
1,Category,undeployed,undeployed,undeployed,undeployed,undeployed,undeployed,CEL Treasury,CEL Users,Institutional Loans,...,NaN,NaN,defi,defi,defi,defi,defi,defi,defi,defi
2,Tier,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,1.0,unknown,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
3,Risk Premium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Default,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,ZRX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0389,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150,ZUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152,check,1,0.0,0.0,NaN,0.0,0.0,0.0,0.0,38,...,0.0,0,0.0,0.0,2.0,1,1,7,NaN,NaN
153,check_account,1,1,1,1.0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,NaN,NaN


In [10]:
'''This is computing the ETHER collateral ratio for Aave and Compounds Celsius Borrows Account'''
defi_main.columns = defi_main.loc[0]
defi_main = defi_main.loc[1:]
borrow_account = defi_main[defi_main['Vault Name'] == 'Celsius Borrows Account']
col_ratio = {}
col_threshold = 2.0
protocols = ['Compound', 'Aave V2']

grouped = borrow_account[borrow_account['Coin']=='ETH'].groupby('Protocol')['sum Original Balance'].sum()
grouped = grouped/grouped.sum()

for protocol in protocols:
    temp = borrow_account[borrow_account['Protocol'] == protocol]
    borrow = sum([bal for bal in temp['sum Balance $USD'] if bal<0])
    try :
        ether = temp[temp['Coin']=='ETH']['sum Balance $USD'].values[0]
    except IndexError:
        ether = 0
    col_ratio[protocol] = max(abs(ether/borrow) - col_threshold,0)
    if ether!=0: col_ratio[protocol] /= abs(ether/borrow)
    try :
        col_ratio[protocol + '_part'] = grouped[protocol]
    except KeyError:
        col_ratio[protocol + '_part'] = 0


In [11]:
# We format the deployable tab
deployable_0 = deployable.copy()
deployable_0 = deployable.iloc[1:,[1,8]]
deployable_0.columns = deployable_0.iloc[0]
deployable_0 = deployable_0.iloc[1:]

In [12]:
# this is generating a assets/liabilities dataframe for waterfall sheet
coin_asset_liability = stats[[0,2,3]].copy()
coin_asset_liability.columns = coin_asset_liability.iloc[0]
coin_asset_liability.drop([0], inplace = True)
coin_asset_liability.dropna(axis = 0 , how = "all", inplace = True)
coin_asset_liability.reset_index(drop = True, inplace = True)
coin_asset_liability.at[0, "Coin/Asset"] = "Total"
coin_asset_liability = coin_asset_liability[coin_asset_liability["Coin/Asset"].notnull()]
coin_asset_liability.columns = ['Coin', 'Net Assets Total', 'Net Liabilities Total']
coin_asset_liability["Net Assets Total"] = coin_asset_liability["Net Assets Total"].astype("float")
coin_asset_liability["Net Liabilities Total"] = coin_asset_liability["Net Liabilities Total"].astype("float")
#coin_asset_liability.columns

In [13]:
# get the coin price from coin recon sheet, there are some coins in FTX which are not listed in the "coin stats" tab
# and have no price. I use FTX api to get their most recent price.
coin_price = stats[[0,1]]
coin_price.columns = ["Coin", "Price"]
coin_price.drop([0], inplace = True)
coin_price.dropna(axis = 0 , how = "all", inplace = True)
coin_price = coin_price[coin_price["Coin"].notnull()]
coin_price.reset_index(drop = True, inplace = True)
coin_price.loc[len(coin_price.index)] = ['Stable Coins', 1] 
srm_price = float(coin_price.loc[coin_price["Coin"] == "SRM", "Price"])
coin_price.loc[len(coin_price.index)] = ['SRM_LOCKED', srm_price] 
#coin_price


existing_coins = coin_price["Coin"].unique()
coins_need_price = ["ATLAS", "SOL","POLIS", "RAY", "SLRS", "BVOL", "YFL", "QI", "CREAM",
                   "FTT", "DASH", "THKD", "ALPHA", "WDGLD", "VSP", "ORBS", "WBTC", 
                   "3CRV", "KIN", "LRC","MSOL"]
needed = []
for coin in coins_need_price:
    if coin not in existing_coins:
        needed.append(coin)
print(needed)
price_dict ={"Coin": [], "Price": []}
for coin in needed:
    url1 = (f"https://ftx.com/api/markets/{coin}/USD")
    data = requests.get(url1).json()
    price_dict["Coin"].append(coin)
    price_dict["Price"].append(data['result']["price"])
needed_price = pd.DataFrame.from_dict(price_dict)
coin_price = pd.concat([coin_price, needed_price])
needed_price

/Users/gabrielchoukroun/.conda/envs/waterfall_python/lib/python3.10/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/Users/gabrielchoukroun/.conda/envs/waterfall_python/lib/python3.10/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


['SLRS', 'BVOL']


,Coin,Price
0,SLRS,0.09279
1,BVOL,556.85000


In [14]:
# Get Risk Premium Info
risk_premium = template[template["Coin"]=="Risk Premium"].T
risk_premium.reset_index(inplace = True)
risk_premium.columns = ["Account", "Risk Premium"]
risk_premium.drop([0], inplace = True)
# there may be duplicate records in risk premium info
risk_premium.drop_duplicates(inplace = True)
# there are some rare occasions that the same account is assinged to two or more different risk premiums, use the first encountered
# the others are likely for some testing purposes, which are usually added to the end
risk_premium = risk_premium[~risk_premium.Account.duplicated()]
risk_premium

,Account,Risk Premium
1,Bank - Balances,NaN
2,Celsius Network Limited (UK),NaN
3,Celsius Network LLC (US),NaN
4,Celsius Network EU UAB (LT),NaN
5,Celsius Network Finance,NaN
...,...,...
253,RD - MARS - TerraUST,NaN
254,DD - Astroport - stLuna,NaN
256,DD - Curve Polygon - EURTUSD,NaN
257,YD - Lido - stETH,NaN


In [15]:
# get coin tier info 
tiers = template[template["Coin"]=="Tier"].T
tiers.reset_index(inplace = True)
tiers.columns = ["Account", "Tier"]
tiers.drop([0], inplace = True)
# there may be duplicate records in tier info
tiers.drop_duplicates(inplace = True)
# there are some rare occasions that the same account is assinged to two or more different tiers, use the first encountered
# the others are likely for some testing purposes, which are usually added to the end 
tiers = tiers[~tiers.Account.duplicated()]
filter4 = tiers['Tier'].isnull()
tiers.at[filter4, "Tier"] = "unassigned"
tiers["Tier"] = tiers["Tier"].astype("string")
#print(len(tiers))
#print(len(tiers[tiers["Tier"] == "unassigned"]))
#print(tiers.Account.nunique())
tiers["Tier"] = tiers["Tier"].apply(lambda x:x.split(".")[0])
#tiers["Tier"].unique()
#tiers

In [16]:
# mapping each accout to a category
categories = template[template["Coin"].isin(["Coin", "Category"])].T
categories.reset_index(drop = True, inplace = True)
categories.columns = ["Account", "Category"]
categories.drop([0], inplace = True)
categories.Category.fillna(value = "unassigned", inplace = True)
#print(len(categories))
#print(len(categories[categories["Category"] == "unassigned"]))
#print(categories.Account.nunique())
#categories

In [17]:
# read the cofa data
cofa_original = pd.read_excel("excel_input/coin_apy_template.xlsx", sheet_name = "COFA")
cofa_original.dropna(axis=0, how='all', inplace=True)
cofa_original.reset_index(drop = True, inplace = True)
#display(cofa_original)
cofa_original.at[cofa_original["Coin"] == "stable (USD)", "Coin"] = "Stable Coins"
cofa_melt_cols = list(cofa_original.columns)
cofa_melt_cols.remove("Coin")
#print(cofa_melt_cols)
cofa = melt(cofa_original, cofa_melt_cols, "Account", "COFA")
cofa = cofa[cofa["COFA"].notnull()]
cofa.reset_index(drop = True, inplace = True)
#cofa

In [18]:
# process the carry tab (like the cofa)
carry.dropna(axis=0, how='all', inplace=True)
carry.reset_index(drop = True, inplace = True)
#display(cofa_original)
carry.at[carry["Coin"] == "stable (USD)", "Coin"] = "Stable Coins"
carry_melt_cols = list(carry.columns)
carry_melt_cols.remove("Coin")
#print(cofa_melt_cols)
carry2 = melt(carry, carry_melt_cols, "Account", "CARRY")
carry2 = carry2[carry2["CARRY"].notnull()]
carry2.reset_index(drop = True, inplace = True)

In [19]:
# process apy data
apy = template[~template["Coin"].isin(["Coin", "Category", "Tier", "Risk Premium"])]
#apy.fillna(value = 0, inplace = True)

# people leave spaces in the excel file sometimes, need to get rid of them
apy.replace([" ", "", "  "], np.nan, inplace = True)
apy =apy[apy["Coin"] != "wBTC (Y/N)"]
#apy

/Users/gabrielchoukroun/.conda/envs/waterfall_python/lib/python3.10/site-packages/pandas/core/frame.py:5238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [20]:
temp = apy[apy['Coin']=='check_account'].iloc[:,1:]
cond = temp>1
duplicated = temp.T[temp.T>1].dropna()

In [21]:
apy2 = apy.copy()
apy2["Coin"] = apy2["Coin"] + "_APY"
apy5 = apy2.T
apy5.reset_index(inplace = True)
apy5.columns = apy5.iloc[0]
apy5.rename(columns = {"Coin": "Account"}, inplace = True)
apy5.drop([0], inplace = True)
#apy5

In [22]:
# convert dyptes to float
for col in apy5.columns:
    if col != "Account":
        apy5.loc[apy5[col].isin(template["Coin"].unique()), col] = np.nan
        apy5[col] = apy5[col].map(lambda x:np.nan if type(x)==str else x)\
            .astype(float)

In [23]:
# get the user collateral and institutional collateral data in a dataframe
# to be uploaded to waterfall sheet 
collateral = stats.copy()
collateral_p1 = collateral.iloc[:, 0].to_frame()
collateral_p1.columns = ["Coin"]
collateral_p2 = collateral.iloc[:, 4:]
collateral_p2.columns = collateral_p2.iloc[2]
collateral = pd.concat([collateral_p1, collateral_p2[["User Collateral", "Inst Collateral"]]], axis = 1)
collateral.drop([0,1,2,3], inplace = True)
collateral = collateral[collateral["Coin"].notnull()]
collateral["User Collateral"] = -1 * collateral["User Collateral"]
collateral["Inst Collateral"] = -1 * collateral["Inst Collateral"]
collateral.fillna(value = 0, inplace = True)
collateral.dtypes

Coin                object
User Collateral    float64
Inst Collateral    float64
dtype: object

In [24]:
# process stats table

# first fill the "asset" or "liability" into row 0
stats.iloc[0] = stats.iloc[0].ffill()
# drop the secondary description of asset or liability (no use)
stats.drop([1], inplace = True)

#split stats into two parts, p1 is coin name and summary, p2 is assets/liabilities
stats_p1 = stats.iloc[:, 0:3]
stats_p1.columns = stats_p1.iloc[0]
stats_p1.reset_index(drop = True, inplace = True)
stats_p1.drop([0, 1, 2], inplace = True)
stats_p1.rename(columns = {"Coin/Asset": "Coin"}, inplace = True)


stats_p2 = stats.iloc[:, 4:]

# filter according to "assets" and the detailed account name cannot be null
stats_p3 = stats_p2.loc[:, stats_p2.loc[2].notnull()]
stats_p4 = stats_p3.loc[:, stats_p3.loc[0] == "Assets"]
stats_p4.reset_index(drop = True, inplace = True)
stats_p4.columns = stats_p4.iloc[1]
stats_p4.reset_index(drop = True, inplace = True)
stats_p4.drop([0, 1, 2], inplace = True)
stats = pd.concat([stats_p1["Coin"], stats_p4], axis = 1)
stats = stats[stats["Coin"].notnull()]
stats

/Users/gabrielchoukroun/.conda/envs/waterfall_python/lib/python3.10/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/Users/gabrielchoukroun/.conda/envs/waterfall_python/lib/python3.10/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Coin,Bank - Balances,Celsius Network Limited (UK),Celsius Network LLC (US),Celsius Network EU UAB (LT),Celsius OTC,Celsius Network Finance,CEL Treasury,CEL Users,Loans Out,...,Mining,EAM - Balances,Others - Asset,BITFINEX,COINBASEPRO,DERIBIT,LIQUID,custodian reserve,Investments,custody!
3,1INCH,0,0,55852.10104,0,0,0,0,0,0,...,0,0,6666666.66,0,0,0,0,18478.20259,0,54433.91
4,3CRV,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,AAVE,0,90.839905,9336.393682,0,0,5.93778,0,0,19620.65,...,0,0,0,0,0,0,0,453.000629,0,2303.51
6,ACA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,ADA,0,16482076.46,15791398.09,1,165526.7501,268.036039,0,0,82136994.07,...,0,0,24330657.36,0.006653,0,0,0,1087908.603,0,6707353.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,yUSD,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
121,yveCRV-DAO,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
122,ZEC,0,560.1715,78015.10272,0,0,0,0,0,13000,...,0,0,0,4630.993209,0,0,0,151423.9166,0,4754.1099
123,ZRX,0,6200000,1020076.29,0,0,272.88,0,0,5100000,...,0,0,0,577785.9306,8735.95141,0,0,-5832.534447,0,217050.15


In [25]:
# process DeFi assets data
defi_p1 = defi.iloc[:, 0:3]
defi_p1.drop([0, 1, 2], inplace = True)
defi_p1.rename(columns = {"Coin/Asset": "Coin"}, inplace = True)
defi_p1
defi_p2 = defi.iloc[:, 3:]

# filter according the detailed account name cannot be null
defi_p3 = defi_p2.loc[:, defi_p2.iloc[1].notnull()]
defi_p3.reset_index(drop = True, inplace = True)
defi_p3.columns = defi_p3.iloc[1]
defi_p3.reset_index(drop = True, inplace = True)
defi_p3.drop([0, 1, 2], inplace = True)
defi_p3
defi = pd.concat([defi_p1["Coin"], defi_p3], axis = 1)
defi = defi[defi["Coin"].notnull()]
#defi

In [26]:
# ADD GAB - SPLIT ON OVERCOLLATERIZED ETH IN CELSIUS BORROWS ACCOUNT
borrow_eth = defi[defi['Coin']=='ETH']['Celsius Borrows Account'].iloc[0]
idx_eth = defi[defi['Coin']=='ETH'].index[0]
for col in ['Aave V2', 'Compound']:
    new_col = f'Overcollaterized - {col}'
    defi[new_col] = 0
    defi.at[idx_eth,new_col] = borrow_eth * col_ratio[col+'_part']* col_ratio[col]
    defi.at[idx_eth,'Celsius Borrows Account'] -= defi.at[idx_eth,new_col]

In [27]:
# process ftx and ftx_cnc summary data
ftx.dropna(axis = 0 , how = "all", inplace = True)
ftx.dropna(axis = 1 , how = "all", inplace = True)
filter_ftx = ((ftx["Total Asset"] == 0) & (ftx["Total Borrows"] == 0))
ftx = ftx[(ftx[ftx.columns[0]].notnull()) & ~filter_ftx]
ftx.drop(columns = ["Total Asset", "Total Borrows"], inplace = True)
ftx_cols = []
for col in ftx.columns:
    ftx_cols.append("FTX - " + col)
ftx.columns = ftx_cols
ftx.rename(columns = {ftx.columns[0]: "Coin"}, inplace = True)

# below lines of code will merge BTC and WBTC together
ftx_btc = ftx[ftx["Coin"].isin(["BTC", "WBTC"])]
ftx_btc = ftx_btc.append(ftx_btc.sum(numeric_only=True), ignore_index=True)
ftx_btc = ftx_btc[~ftx_btc["Coin"].isin(["BTC", "WBTC"])]
ftx_btc.reset_index(drop = True, inplace = True)
ftx_btc.loc[0, "Coin"] = "BTC"
ftx = pd.concat([ftx_btc, ftx[~ftx["Coin"].isin(["BTC", "WBTC"])]])

#ftx_cnc
ftx_cnc.dropna(axis = 0 , how = "all", inplace = True)
ftx_cnc.dropna(axis = 1 , how = "all", inplace = True)
filter_ftx = (ftx_cnc["NET Size"] == 0)
ftx_cnc = ftx_cnc[(ftx_cnc[ftx_cnc.columns[0]].notnull()) & ~filter_ftx]
ftx_cols = []
for col in ftx_cnc.columns:
    ftx_cols.append("FTX - " + col)
ftx_cnc.columns = ftx_cols
ftx_cnc.rename(columns = {ftx_cnc.columns[0]: "Coin", ftx_cnc.columns[1]:'Future'}, inplace = True)

In [28]:
# get update date and time
now = datetime.now()
dt_string = now.strftime("%Y-%m-%d %H:%M:%S")
#print("Updated at:", dt_string)

In [29]:
# define the stable coins, these coins are listed together under "stable coins" in waterfall sheet
stables = ['alUSD','BUSD', 'GUSD', 'LUSD', 'LUSD Curve','MCDAI', 'PAX', 'SUSD', 'TUSD', 
           'USDC', 'USDT ERC20', 'ZUSD', "USD"]

In [30]:
# create a Tier dataframe for merging purpose
tier_dict = {"Tier":["1", "2", "3", "4", "5", "unassigned"]}
coin_tiers = pd.DataFrame.from_dict(tier_dict)
coin_tiers

,Tier
0,1
1,2
2,3
3,4
4,5
5,unassigned


In [31]:
# merge 3 dataframes together, coin stats/Defi assets/FTX summary
coin_stats = stats.merge(defi, on = "Coin", how = "outer")
coin_stats = coin_stats.merge(ftx, on = "Coin", how = "outer")
#coin_stats = coin_stats.merge(ftx_cnc, on='Coin', how='outer')
coin_stats.fillna(value = 0, inplace = True)
coin_stats_col = []
for col in coin_stats.columns:
    coin_stats_col.append(col.strip())
coin_stats.columns = coin_stats_col
len(coin_stats.columns)
coin_stats

,Coin,Bank - Balances,Celsius Network Limited (UK),Celsius Network LLC (US),Celsius Network EU UAB (LT),Celsius OTC,Celsius Network Finance,CEL Treasury,CEL Users,Loans Out,...,FTX - CEL,FTX - CnC,FTX - DeFi,FTX - Grayscale,FTX - Jacob,FTX - Main Account,FTX - TEAM Directional,FTX - Test Test_,FTX - cel_staking,FTX - Management
0,1INCH,0.0,0.000000e+00,5.585210e+04,0.0,0.0000,0.000000,0.0,0.0,0.00,...,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.000,0.0
1,3CRV,0.0,0.000000e+00,0.000000e+00,0.0,0.0000,0.000000,0.0,0.0,0.00,...,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.000,0.0
2,AAVE,0.0,9.083990e+01,9.336394e+03,0.0,0.0000,5.937780,0.0,0.0,19620.65,...,0.00000,0.0,-0.00474,0.0,16803.82769,0.0,0.0,0.0,0.000,0.0
3,ACA,0.0,0.000000e+00,0.000000e+00,0.0,0.0000,0.000000,0.0,0.0,0.00,...,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.000,0.0
4,ADA,0.0,1.648208e+07,1.579140e+07,1.0,165526.7501,268.036039,0.0,0.0,82136994.07,...,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,yveCRV-DAO,0.0,0.000000e+00,0.000000e+00,0.0,0.0000,0.000000,0.0,0.0,0.00,...,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.000,0.0
119,ZEC,0.0,5.601715e+02,7.801510e+04,0.0,0.0000,0.000000,0.0,0.0,13000.00,...,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.000,0.0
120,ZRX,0.0,6.200000e+06,1.020076e+06,0.0,0.0000,272.880000,0.0,0.0,5100000.00,...,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.000,0.0
121,ZUSD,0.0,0.000000e+00,7.911896e+04,0.0,0.0000,0.000000,0.0,0.0,0.00,...,0.00000,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.000,0.0


In [32]:
# get a list of coins in the above dataframe and add "Stable Coins" into that list
coin_list = list(coin_stats["Coin"].unique())
coin_list.append("Stable Coins")
#coin_list

In [33]:
acc_list = np.asarray(tiers["Account"])
len(acc_list)

256

In [34]:
# this block and the following block are meant to show which accounts from coin recon sheet are not listed 
# in tier/apy sheet, or which accounts listed in the tier/apy data no longer exists in coin recon sheet
not_covered = []
for col in coin_stats.columns:
    if col not in acc_list:
        not_covered.append(col)
not_covered    

['Coin',
 'DeFi Borrows - Collateral',
 'DeFi Borrows - Tokens',
 'DeFi Assets - Assets',
 'FTX',
 'custodian reserve',
 'custody!',
 'DD - Vesper - AVAX',
 'CEL Buying activity',
 'Clearpool Finance Lending',
 'Deployment - Vesper (OLD DO NOT USE)',
 'SGB Staking',
 'dd-cx-OLIVE-ETH',
 'dd-cx-Olive-USDC',
 'FTX - DeFi Perpswap',
 'FTX - DEFI_CORE_MARGIN_FTX',
 'FTX - DEFI_REWARDFOCUS_FTX',
 'FTX - DEFI-Margin',
 'FTX - LENDING_SPOTLEND_FTX',
 'FTX - Test Test_']

In [35]:
not_covered = []
for col in acc_list:
    if col not in coin_stats.columns:
        not_covered.append(col)
not_covered

['Impermanent_loss_hedge__RonSabo',
 'Hedge_Options',
 'FTX - Kairon',
 'FTX - Directional Trading 2',
 'FTX - Johannes',
 'OKEX',
 'Deployment - Vesper',
 'YD - Compound - WBTC',
 'BlockDemon ETH Staking',
 'DD - Vesper - AVAX\n',
 'Deployment - Stable Coin Swaps',
 'Deployment Team - COMP supply',
 'Deployment- 1INCH Staking',
 'Convex: cvxBUSD3CRV-f',
 'FTX - LONG1',
 'YD - Curve - SBTC',
 'YD - Keeper - renBTC',
 'MATIC Staking 7',
 'Deployment - 1INCH Staking (testing)',
 'FTX - Kairon2',
 'YD - Curve - alUSD',
 'DD - SYNAPSIS - WETH/Arbitrum',
 'DD - SYNAPSIS - WETH/Optimism',
 'DD - SYNAPSIS - USD/Avalanche',
 'DD - SYNAPSIS - USD/Polygon',
 'DD - Benqi - Avalanche',
 'SGB',
 'DD-Anchor-Terra',
 'SGB Staking 2',
 'FTX - CEFI_TACTICAL_STRAT2_FTX',
 'FTX - TREASURY_EXCHANGELEND_SPOTLEND_FTX',
 'CEL TBD',
 'castody',
 'RD - MARS - TerraUST',
 'DD - Astroport - stLuna']

In [36]:
# transpose coin_stats to get it in right shape
account_toremove = ['DeFi Borrows - Assets', 'DeFi Assets - Assets', "FTX",
                   "DeFi Borrows - Collateral", "DeFi Borrows - Tokens"]
coin_stats_t = coin_stats.T
new_header = coin_stats_t.iloc[0] #grab the first row for the header
coin_stats_t = coin_stats_t[1:] #take the data less the header row
coin_stats_t.columns = new_header #set the header row as the df header
coin_stats_t.reset_index(inplace = True)
coin_stats_t.rename(columns={coin_stats_t.columns[0]: "Account" }, inplace = True)
coin_stats_t.reset_index(drop=True, inplace = True)
#coin_stats_t = coin_stats_t[coin_stats_t["Account"] != "Defi Assets"]
coin_stats_t = coin_stats_t[~coin_stats_t["Account"].isin(account_toremove)]


In [37]:
coin_stats_t

Coin,Account,1INCH,3CRV,AAVE,ACA,ADA,agEUR,ALCX,ALICE,ALPHA,...,XRP,XTZ,YFI,YFL,yUSD,yveCRV-DAO,ZEC,ZRX,ZUSD,SRM_LOCKED
0,Bank - Balances,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Celsius Network Limited (UK),0.0,0.0,90.839905,0.0,16482076.46,0.0,0.0,0.0,0.0,...,444.249292,5075.0699,0.2277,0.0,0.0,0.0,560.1715,6200000.0,0.0,0.0
2,Celsius Network LLC (US),55852.10104,0.0,9336.393682,0.0,15791398.09,0.0,0.0,0.0,0.0,...,236657.1217,179543.9791,1.808279,0.0,0.0,0.0,78015.10272,1020076.29,79118.96141,0.0
3,Celsius Network EU UAB (LT),0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,7380760.68,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Celsius OTC,0.0,0.0,0.0,0.0,165526.7501,0.0,0.0,0.0,0.0,...,61107.73346,10020.4335,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,FTX - Main Account,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
236,FTX - TEAM Directional,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
237,FTX - Test Test_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
238,FTX - cel_staking,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2857812.931


In [38]:
# adding Tier info
coin_stat_tier = tiers.merge(coin_stats_t, on = "Account", how = "right")
#coin_stat_tier["Tier"].fillna(value = "unknown", inplace = True)

coin_stat_tier['Stable Coins']= coin_stat_tier[stables].sum(axis=1)
#coin_stat_tier.to_excel("excel_outputs/coin_stats_tier_live.xlsx", index = False)

#print(len(coin_stat_tier[coin_stat_tier["Tier"] == "unknown"]))
coin_stat_tier["Tier"] = coin_stat_tier["Tier"].astype("str")

In [39]:
# to calculate stable coin apy, we need to calculate for each stable coin yield = # of each stable coin * apy of each stable
# and stable coin apy = sum of all stable coin yiled / total # of stable coins
# this prepares a list of columns needed to be summed.
stable_yield = []
for coin in stables:
    stable_yield.append(coin+"_yield")
stable_yield

['alUSD_yield',
 'BUSD_yield',
 'GUSD_yield',
 'LUSD_yield',
 'LUSD Curve_yield',
 'MCDAI_yield',
 'PAX_yield',
 'SUSD_yield',
 'TUSD_yield',
 'USDC_yield',
 'USDT ERC20_yield',
 'ZUSD_yield',
 'USD_yield']

In [40]:
# ADD --------------------------
# ------------------------------
# I need to add a function that will fill _APY columns with the default values
def default_fill(row):
    if ~ np.isnan(row['Default_APY']): 
        row.loc[apy_cols] = row[apy_cols].fillna(row['Default_APY'])
    return row

In [41]:
# calculate the yield = # of coins * apy for each coin
coin_stat_tier_apy = coin_stat_tier.merge(apy5, on="Account", how="left")
# coin_stat_tier_apy.fillna(value = 0, inplace = True)

# --------- ADD GAB - Replace NA APY by the corresponding Default APY values if it exists -------------------
apy_cols = ['_APY' in col for col in coin_stat_tier_apy.columns]
coin_stat_tier_apy = coin_stat_tier_apy.apply(default_fill, axis=1, result_type='broadcast')
coin_stat_tier_apy['Stable Coins'] = coin_stat_tier_apy['Stable Coins'].astype(np.float64)
coin_stat_tier_apy = coin_stat_tier_apy.loc[:, ~coin_stat_tier_apy.columns.duplicated()]
# coin_stat_tier_apy.to_excel("C:/celsius/Liquidity/coin_stat_tier_apy.xlsx", index = False)
for coin in coin_list:
    # print(coin)
    yield_name = coin + "_yield"
    coin_apy = coin + "_APY"
    if coin_apy in coin_stat_tier_apy.columns:
        coin_stat_tier_apy[yield_name] = coin_stat_tier_apy[coin] * coin_stat_tier_apy[coin_apy]
    else:
        coin_stat_tier_apy[yield_name] = 0
        coin_stat_tier_apy[coin_apy] = np.nan
coin_stat_tier_apy['Stable Coins_yield'] = coin_stat_tier_apy[stable_yield].sum(axis=1)
coin_stat_tier_apy['Stable Coins_APY'] = coin_stat_tier_apy['Stable Coins_yield'] / coin_stat_tier_apy[
    'Stable Coins']

/var/folders/q7/qhj4q23x49vb9f__hlp8t3280000gn/T/ipykernel_53933/2925171909.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  coin_stat_tier_apy[yield_name] = coin_stat_tier_apy[coin] * coin_stat_tier_apy[coin_apy]
/var/folders/q7/qhj4q23x49vb9f__hlp8t3280000gn/T/ipykernel_53933/2925171909.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  coin_stat_tier_apy[yield_name] = 0
/var/folders/q7/qhj4q23x49vb9f__hlp8t3280000gn/T/ipykernel_53933/2925171909.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usuall

In [42]:
apy_cols = ['_APY' in col for col in coin_stat_tier_apy.columns]
coin_stat_tier_apy.loc[:,apy_cols].head()

,Default_APY,1INCH_APY,am3CRV_APY,3CRV_APY,AAVE_APY,ADA_APY,agEUR_APY,ALCX_APY,ALICE_APY,ALPHA_APY,...,check_account_APY,All Coins_APY,ACA_APY,BETH_APY,cxBTC_APY,DOGE_APY,KIN_APY,MSOL_APY,QRDO_APY,Stable Coins_APY
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [43]:
# for each coin, groupy by tier so we have a dataframe containing coin level tier data
# also calculating the apy for each coin on tier level
for coin in coin_list:
    #print(coin)
    coin_yield = coin+"_yield"
    coin_apy = coin+"_APY"
    df1 = coin_stat_tier_apy[[coin, coin_yield, "Tier"]]
    df2 = df1.groupby('Tier', as_index = False).agg({coin: "sum",
                                                    coin_yield: "sum"})
    df2[coin_apy] = df2[coin_yield]/ df2[coin]
    df2.drop(columns = [coin_yield], inplace = True)
    #display(df2)
    coin_tiers = coin_tiers.merge(df2, on = "Tier", how = "left")
coin_tiers.fillna(value = 0, inplace = True)
#coin_tiers

In [44]:
coin_tier_p1_cols = ["Tier"]
coin_tier_p2_cols = ["Tier"]
filter2 = coin_tiers["Tier"] != "unassigned"
for col in coin_tiers.columns:
    if col != "Tier" and "_APY" in col:
        coin_tier_p2_cols.append(col)
    elif col != "Tier":
        coin_tier_p1_cols.append(col)
    else:
        pass
#print(len(coin_tier_p1_cols))
#len(coin_tier_p2_cols)

In [45]:
coin_tier_p1 = coin_tiers[filter2][coin_tier_p1_cols].T
coin_tier_p1.reset_index(inplace = True)
coin_tier_p1.columns = coin_tier_p1.iloc[0]
coin_tier_p1.drop([0], inplace = True)
new_name = ["Coin"]
for col in coin_tier_p1.columns:
    if col != "Tier":
        new_name.append("Coin_Tier_"+str(col)[0])
coin_tier_p1.columns = new_name
#coin_tier_p1

In [46]:
coin_tier_p2 = coin_tiers[filter2][coin_tier_p2_cols]
coin_tier_p2.columns = coin_tier_p1_cols
coin_tier_p2 = coin_tier_p2.T
coin_tier_p2.reset_index(inplace = True)
coin_tier_p2.columns = coin_tier_p2.iloc[0]
coin_tier_p2.drop([0], inplace = True)
new_name = ["Coin"]
for col in coin_tier_p2.columns:
    if col != "Tier":
        new_name.append("APY_Tier_"+str(col)[0])
coin_tier_p2.columns = new_name
#coin_tier_p2

In [47]:
coin_tiers = coin_tier_p1.merge(coin_tier_p2, on = "Coin", how = "inner")
filter3 = ~coin_tiers["Coin"].isin(stables)
#coin_tiers.columns

In [48]:
coin_stat_tier_apy[coin_stat_tier_apy['Account'] == 'DD - Badger - tBTC']

,Account,Tier,1INCH,3CRV,AAVE,ACA,ADA,agEUR,ALCX,ALICE,...,YFI_yield,YFL_yield,yUSD_yield,yveCRV-DAO_yield,ZEC_yield,ZRX_yield,ZUSD_yield,SRM_LOCKED_yield,Stable Coins_yield,Stable Coins_APY


## the below code is to generat the pivot_data tab

In [49]:
tier_override

,0,1,2,3,4,5,6,7,8
0,Coin,Account,Tier 1,Tier 2,Tier 3,Tier 4,Tier 5,Tier 6,Check
1,MATIC,Direct Staking,0.0,0.0,1.0,0.0,0.0,0.0,True
2,ADA,Direct Staking,0.0,1.0,0.0,0.0,0.0,0.0,True
3,WBTC,DD - Benqi - Avalanche,0.5,0.0,0.0,0.0,0.0,0.5,True
4,USDC,YD - AlphaHv2- USDC.e,0.0,1.0,0.0,0.0,0.0,0.0,True
...,...,...,...,...,...,...,...,...,...
449,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
451,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
452,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [50]:
tier_override.columns = tier_override.loc[0]
tier_override = tier_override.loc[1:]
tier_override.dropna(subset=['Coin'], inplace=True)
tier_override.fillna(0, inplace=True)
tier_override = tier_override[tier_override['Check']==True]
tier_override

/Users/gabrielchoukroun/.conda/envs/waterfall_python/lib/python3.10/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/Users/gabrielchoukroun/.conda/envs/waterfall_python/lib/python3.10/site-packages/pandas/core/frame.py:5176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


,Coin,Account,Tier 1,Tier 2,Tier 3,Tier 4,Tier 5,Tier 6,Check
1,MATIC,Direct Staking,0.0,0.0,1.0,0.0,0.0,0.0,True
2,ADA,Direct Staking,0.0,1.0,0.0,0.0,0.0,0.0,True
3,WBTC,DD - Benqi - Avalanche,0.5,0.0,0.0,0.0,0.0,0.5,True
4,USDC,YD - AlphaHv2- USDC.e,0.0,1.0,0.0,0.0,0.0,0.0,True
5,USDT,YD - AlphaHv2- USDC.e,0.0,1.0,0.0,0.0,0.0,0.0,True
...,...,...,...,...,...,...,...,...,...
282,ETH,Direct Staking,0.0,0.0,0.0,0.0,0.0,1.0,True
283,DASH,Direct Staking,0.0,1.0,0.0,0.0,0.0,0.0,True
284,SOL,Kraken Staking,1.0,0.0,0.0,0.0,0.0,0.0,True
285,EURT,DD - Curve Polygon - EURTUSD,0.0,0.0,1.0,0.0,0.0,0.0,True


In [51]:
# Category Override
category_override.columns = category_override.loc[0]
category_override = category_override.loc[1:]
category_override.dropna(subset=['Coin'], inplace=True)
category_override.fillna(0, inplace=True)
category_override

,Coin,Account,Category
1,MATIC,BITFINEX,Staking
2,ETH,Stakehound,Other


In [52]:
# Create df1 dataframe
df2 = pd.DataFrame()
some_dict = {"Coin":[], "Category":[], "Account": [], "Tier": [],
             "# of Coins": [], "APY": []}
df1 = pd.DataFrame.from_dict(some_dict)
col_names = ["Coin", "Category", "Account", "Tier", "# of Coins",
             "APY", "COFA", "USD Value", "USD Value * COFA",
             "USD Value * APY", "Deployment Status", "Manual", "CARRY",
             "USD Value * CARRY", "Original Coin", "Risk Premium", "USD Value * Risk Premium",
             "Credited Carry", "Amount Received $ * Credited Carry"]
df1

,Coin,Category,Account,Tier,# of Coins,APY


In [53]:
# Fill df1 Dataframe coin by coin
for coin in coin_list:
    #if coin not in stables:
    used_cols = ["Account", "Tier", coin, coin+"_APY"]
    df2 = categories.merge(coin_stat_tier_apy[used_cols], on = "Account", how = "right")
    df2["Coin"] = coin
    df2.rename(columns = {coin: "# of Coins", coin+"_APY": "APY"}, inplace = True)
    df1 = pd.concat([df1, df2])

filter1 = df1["# of Coins"] > 10e-6
df1 = df1[filter1]
cats = list(df1["Category"].unique())
accs = list(df1["Account"].unique())
cofa_accts = list(cofa["Account"].unique())
cofa_accts.remove("Default")
cofa_accts.remove('FTX - CEL')
carry_accts = list(carry2["Account"].unique())
#carry_accts.remove("Default")
#cofa_accts

In [54]:
# Add CnC
# Positions
pivot_data_cnc = pd.DataFrame()
cnc_accounts = ['CEFI_YIELD_CNC1_FTX', 'CEFI_TACTICAL_STRAT1_FTX']
cnc_accounts = ['FTX - ' + account for account in cnc_accounts]
for account in cnc_accounts:
    pivot_cnc = ftx_cnc[['Coin','Future'] + [account]]
    filter = pivot_cnc[account] !=0
    pivot_cnc = pivot_cnc[filter]
    pivot_cnc_apy = pivot_cnc.merge(cnc_apy, on='Coin', how='left')
    pivot_cnc_apy.columns = ['Coin','Original Coin', '# of Coins', 'APY']
    pivot_cnc_apy['Account'] = account
    # Handle Short vs Long positions
    short_filter = pivot_cnc_apy['# of Coins']>0
    pivot_cnc_apy.loc[short_filter, 'APY'] = -pivot_cnc_apy.loc[short_filter, 'APY']

    long_filter = pivot_cnc_apy['# of Coins']<0
    pivot_cnc_apy.loc[long_filter, 'Original Coin'] = pivot_cnc_apy.loc[long_filter, 'Coin']
    pivot_cnc_apy.loc[long_filter, 'Coin'] = 'USD'
    #Add tier
    pivot_cnc_apy_tier = pivot_cnc_apy.merge(tiers, on='Account', how='left')
    pivot_cnc_apy['# of Coins'] = pivot_cnc_apy['# of Coins']
    pivot_data_cnc = pd.concat([pivot_data_cnc,pivot_cnc_apy_tier])
pivot_data_cnc['Category'] = 'Exchange'

In [55]:
# Add cofa
def add_cofa_carry(df1, cofa, cofa_accts, tab="COFA"):
    default_cofa = cofa[cofa["Account"] == "Default"][["Coin", tab]]
    df2 = df1.merge(default_cofa, on = "Coin", how = "left")
    for col in cofa_accts:
        #print(col)
        cofa_2 = cofa[cofa["Account"] == col]
        if col in cats:
            cofa_2.columns = ["Coin", "Category", f"{tab}-2"]
            df2 = df2.merge(cofa_2, on=["Coin", "Category"], how='left')
        elif col in accs:
            cofa_2.columns = ["Coin", "Account", f"{tab}-2"]
            df2 = df2.merge(cofa_2, on=["Coin", "Account"], how='left')
        else:
            continue
        df2_p1 = df2[df2[f"{tab}-2"].isnull()].drop(columns = [f"{tab}-2"])
        df2_p2 = df2[df2[f"{tab}-2"].notnull()]
        df2_p2[tab] = df2_p2[f"{tab}-2"]
        df2_p2.drop(columns = [f"{tab}-2"], inplace = True)
        df2 = pd.concat([df2_p1, df2_p2])
        df2.sort_values(by = ["Coin"], inplace = True)
    return df2
df2 = add_cofa_carry(df1, cofa, cofa_accts)
pivot_data_cnc2 = add_cofa_carry(pivot_data_cnc, cofa, cofa_accts)
# STABLE COFA
#assert False
cond = (df2['Coin'].isin(stables)) & (df2['Account'].isin(cofa_accts))
df2.loc[cond, 'COFA'] = df2.loc[cond, 'COFA'].fillna(0)
cond = (cofa['Coin'].isin(['Stable Coins'])) & (cofa['Account'].isin(['Default']))
stable_cofa = cofa.loc[cond,'COFA'].values[0]
df2.loc[df2['Coin'].isin(stables), 'COFA'] = df2.loc[df2['Coin'].isin(stables), 'COFA'].fillna(stable_cofa)

# CONCAT cnc2 to df2
df2['Original Coin'] = df2['Coin']
df2 = pd.concat([df2,pivot_data_cnc2])

/var/folders/q7/qhj4q23x49vb9f__hlp8t3280000gn/T/ipykernel_53933/4063274540.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_p2[tab] = df2_p2[f"{tab}-2"]
/Users/gabrielchoukroun/.conda/envs/waterfall_python/lib/python3.10/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [56]:
# Add Manual Adjustements
manual_adjustements['Manual'] = True
df2['Manual'] = False
df2 = pd.concat([df2,manual_adjustements]).reset_index(drop=True)

In [57]:
assert False

AssertionError: 

In [ ]:
 # Add Deployment Status
df2 = df2.drop_duplicates()

deployable_0.columns = ['Coin', 'Total Undeployed']
deployable_0['Total Undeployed'] = deployable_0['Total Undeployed'].map(lambda x: 0 if x=='#REF!' else x)
deployable_0.drop_duplicates(inplace=True)
first_accounts = ['Celsius Network Limited (UK)', 'Celsius Network LLC (US)']

total_stable = deployable_0[deployable_0['Coin'].isin(stables)]['Total Undeployed'].sum()
deployable_0 = deployable_0.append({'Coin': 'Stable Coins',
                                    'Total Undeployed': total_stable}, ignore_index=True)
df2['Deployment Status'] = 'Deployed'
df2.loc[df2['Category'].isin(['undeployed', 'Undeployed']),'Deployment Status'] = 'Liquidity Reserve'
undeployed_accounts = df2[df2['Category'].isin(['undeployed', 'Undeployed'])]['Account'].unique().tolist()
# UK First than US
first_accounts = ['Celsius Network Limited (UK)', 'Celsius Network LLC (US)']
undeployed_accounts = [x for x in undeployed_accounts if x not in (first_accounts)]
undeployed_accounts = first_accounts + undeployed_accounts

# undeployed_accounts = np.unique(first_accounts + undeployed_accounts)
df2['# of Coins'].fillna(0, inplace=True)
for account in undeployed_accounts:
    for coin in deployable_0['Coin']:
        cond1 = (df2['Account'] == account) & (df2['Coin'] == coin)
        cond2 = deployable_0['Coin'] == coin
        if sum(cond1) + sum(cond2) < 2: continue

        if df2[cond1]['# of Coins'].values < deployable_0[cond2]['Total Undeployed'].values:
            deployable_0.loc[cond2, 'Total Undeployed'] -= df2.loc[cond1, '# of Coins'].values
            df2.loc[cond1, 'Deployment Status'] = 'Deployable'
        else:
            df2.loc[cond1, '# of Coins'] -= deployable_0.loc[cond2, 'Total Undeployed'].values
            df2.loc[cond1, 'Deployment Status'] = 'Liquidity Reserve'
            new_row = df2.loc[cond1]
            if deployable_0.loc[cond2, 'Total Undeployed'].values > 0:
                new_row['# of Coins'] = deployable_0.loc[cond2, 'Total Undeployed'].values
                new_row['Deployment Status'] = 'Deployable'
                df2 = df2.append(new_row, ignore_index=True)
                deployable_0.loc[cond2, 'Total Undeployed'] = 0

df2.loc[df2['Category'].isin(['underdeployed','Underdeployed']), 'Deployment Status'] = 'Deployable'
df2['Category'] = df2['Category'].str.strip()
df2.loc[df2['Category'].isin(['Other', 'Mining']), 'Deployment Status'] = 'Investments'

In [ ]:
# Tier Override
df2_override = df2.copy()

for row in tier_override.iterrows():
    tiers_col = [f"Tier {i}" for i in range(1,7)]
    coin = row[1]['Coin']
    account = row[1]['Account']
    cond1 = (df2_override['Coin'].isin([coin])) & (df2_override['Account'].isin([account]))
    if sum(cond1)>0:
        pivot_row = df2_override[cond1]
        old_tier = pivot_row['Tier'].values[0]
        for i,col in enumerate(tiers_col):
            i+=1
            new_row = pivot_row.copy()
            tier_ratio = row[1][col]
            if (tier_ratio>0):
                new_row['Tier'] = i
                new_row['# of Coins'] = new_row['# of Coins'].values[0]*tier_ratio
                df2_override = pd.concat([df2_override,new_row]).reset_index(drop=True)

        cond1 = (df2_override['Coin'].isin([coin])) & (df2_override['Account'].isin([account]))
        cond2 = cond1 & (df2_override['Tier']==old_tier)
        if sum(cond2)==1:
            df2_override.drop(df2_override[cond2].index, inplace=True)
        df2_override.drop_duplicates(inplace=True)
        assert df2_override[cond1]['# of Coins'].sum(axis=0) == pivot_row['# of Coins'].values[0]

df2 = df2_override

In [ ]:
# Updating Institutional Loan APY
loan_apy = loan_book[['loan_asset_short','loan_amount','interest_rate']]
loan_total = loan_apy.groupby('loan_asset_short')['loan_amount'].sum().reset_index()
loan_apy = loan_apy.merge(loan_total, on='loan_asset_short', how='left')
loan_apy['loan_amount_%'] = loan_apy['loan_amount_x']/loan_apy['loan_amount_y']
loan_apy['ir X loan_amount_%'] = loan_apy['loan_amount_%'] * loan_apy['interest_rate']
loan_apy2 = loan_apy.groupby('loan_asset_short')['ir X loan_amount_%'].sum().reset_index()
loan_apy2.columns = ['Coin', 'APY']

for coin in loan_apy2['Coin'].unique():
    apy_coin = loan_apy2[loan_apy2['Coin']==coin]['APY'].values[0]/100
    cond = (df2['Category']=='Institutional Loans') & (df2['Coin']==coin)
    df2.loc[cond, 'APY'] = apy_coin

In [ ]:
# Move EFH loan to other
to_move = loan_book[loan_book['borrower_id']=='bc55708f-e65c-4654-8410-6f70bfd1645f'].groupby('loan_asset_short')['loan_amount'].sum().reset_index()
for coin in to_move['loan_asset_short'].unique():
    cond = (df2['Account']=='Loans Out') & (df2['Coin']==coin)
    new_row = df2[cond]
    df2.loc[cond,'# of Coins'] -= to_move[to_move['loan_asset_short']==coin]['loan_amount'].values[0]
    new_row['Category'] = 'Other'
    new_row['# of Coins'] = to_move[to_move['loan_asset_short']==coin]['loan_amount'].values[0]
    new_row['APY'] = 0
    df2 = pd.concat([df2,new_row])



In [ ]:
#df2.to_excel("excel_outputs/cofa_2.xlsx", index = False)
# Add Price
df2 = df2.merge(coin_price, on = "Coin", how = "left")
df2["USD Value"] = df2["# of Coins"] * df2["Price"]
df2["USD Value * APY"] = df2["USD Value"] * df2["APY"]
df2["Tier"] = df2["Tier"].astype(str).apply(lambda x:x.split(".")[0])

df2["USD Value * COFA"] = df2["USD Value"] * df2["COFA"]
df2.sort_values(by = "USD Value", ascending = False, inplace = True)
df2['Category'] = df2['Category'].str.strip()
df2

In [ ]:
# EDIT LOAN BOOK TO COMPUTE CARRY PER ASSET
loan_book_risk['collateral_asset_short'] = loan_book_risk['collateral_asset_short'].str.replace('mSOL','MSOL')
loan_book2 = loan_book_risk.merge(coin_price,left_on='loan_asset_short', right_on='Coin', how='left')
loan_book2.drop('Coin', axis=1,inplace=True)
loan_book2.columns = loan_book2.columns[:-1] .tolist() + ['price_asset']
loan_book3 = loan_book2.merge(coin_price,left_on='collateral_asset_short', right_on='Coin', how='left')
loan_book3.drop('Coin', axis=1,inplace=True)
loan_book3.columns = loan_book3.columns[:-1].tolist() + ['price_collateral']
loan_book3.loc[loan_book3['loan_asset_short']=='WETH', 'price_asset'] =  coin_price.loc[coin_price['Coin']=='ETH','Price'].values[0]

loan_book3['loan_amount $'] = loan_book3['loan_amount'] * loan_book3['price_asset']
loan_book3['collateral_amount $'] = loan_book3['collateral_amount'] * loan_book3['price_collateral']
total_collateral = loan_book3.groupby('loan_asset_short')['collateral_amount $'].sum().reset_index()
loan_carry = loan_book3.merge(carry2[carry2['Account']=='Default'], left_on='collateral_asset_short', right_on='Coin')
loan_carry['collateral_amount $ x Carry'] = loan_carry['collateral_amount $'] * loan_carry['CARRY']


grouped = loan_carry.groupby('loan_asset_short')[['collateral_amount $ x Carry','loan_amount $']].sum()
grouped['CARRY'] = grouped['collateral_amount $ x Carry']/grouped['loan_amount $']
grouped.drop(['collateral_amount $ x Carry','loan_amount $'], axis=1, inplace=True)
grouped['Account'] = 'Loans Out'
grouped.reset_index(inplace=True)
cols = ['Coin', 'CARRY', 'Account']
grouped.columns = cols
grouped = grouped[carry2.columns]
carry3 = pd.concat([carry2,grouped])

loan_book4 = loan_book3.merge(carry3[carry3['Account']=='Loans Out'], left_on='loan_asset_short', right_on='Coin',how='left')
loan_book4.drop(['Account','Coin', 'Company Name'],axis=1,inplace=True)
loan_book4.columns = loan_book4.columns.tolist()[:-1] + ['Carry_loan_asset']

In [ ]:
#ADD CARRY
carry_accts += ['Loans Out']
df2 = add_cofa_carry(df2, carry2, carry_accts, tab='CARRY')
try:
    default = carry2.loc[carry2['Coin'].isin(['Stable Coins']) & (carry2['Account'].isin(['Default'])),'CARRY'].values[0]
except IndexError:
    default = np.nan
# STABLE CARRY
for account in carry_accts:
    cond = (carry2['Coin'].isin(['Stable Coins'])) & (carry2['Account'].isin([account]))
    if account in carry2.loc[carry2['Coin'].isin(['Stable Coins'])]['Account'].unique():
        stable_carry = carry2.loc[cond,'CARRY'].values[0]
    else:
        stable_carry = default

    cond = (df2['Coin'].isin(['Stable Coins'])) & (df2['Account'].isin([account]))
    if sum(cond)>1:
        df2.loc[cond, 'CARRY'] = stable_carry

# USD VALUE * CARRY
df2.loc[(df2['Coin'].isin(['Stable Coins'])), 'CARRY'] = df2.loc[(df2['Coin'].isin(['Stable Coins'])), 'CARRY'].fillna(default)
df2["USD Value * CARRY"] = df2["USD Value"] * df2["CARRY"]
df2.sort_values(by = "USD Value", ascending = False, inplace = True)

# Credited Carry for Institutional Loans
df2['Credited Carry'] = np.nan
df2['Amount Received $ * Credited Carry']= np.nan
credited_carry = loan_book4[['loan_asset_short','Carry_loan_asset']].drop_duplicates()
for coin in credited_carry['loan_asset_short']:
    credited = credited_carry[credited_carry['loan_asset_short']==coin]['Carry_loan_asset'].values[0]
    df2.loc[(df2['Coin']==coin)&(df2['Category']=='Institutional Loans'), 'Credited Carry'] = credited
    df2.loc[(df2['Coin']==coin)&(df2['Category']=='Institutional Loans'), 'Amount Received $ * Credited Carry'] = total_collateral[total_collateral['loan_asset_short']==coin]['collateral_amount $'].values[0] * credited

# Credited Carry for CnC accounts
for account in cnc_accounts:
    col_ratio = 1
    cond = (df2['Original Coin'].map(lambda x: 'PERP' in x)) & (df2['Account']==account)
    df2.loc[cond, 'Credited Carry'] = df2.loc[cond, 'CARRY']
    df2.loc[cond,'Amount Received $ * Credited Carry'] = abs(col_ratio * df2.loc[cond, 'USD Value'] * df2.loc[cond,'Credited Carry'])
    # If Shorted Collateral - USD
    df2.loc[cond & (df2['# of Coins']>0), 'Original Coin'] = df2.loc[cond & (df2['# of Coins']>0), 'Coin']
    df2.loc[cond & (df2['# of Coins']>0), 'Coin'] = 'USD'

In [ ]:
loan_book4[loan_book4['borrower_id']=='bc55708f-e65c-4654-8410-6f70bfd1645f']

In [ ]:
df2[df2['Account']=='Loans Out']

In [ ]:
# EDIT FTX JACOB Account
cond = (df2['Account'] == 'FTX - Jacob') & (df2['APY']>df2['CARRY'])
df2.loc[cond, "Category"] = "Exchange"

In [ ]:
# EDIT Original Coin
cond = df2['Coin'] == 'Stable Coins'
df2.drop(df2[cond].index, axis=0, inplace=True)
df2['Coin'] = df2['Coin'].replace('WBTC', 'BTC')
for coin in stables:
    df2['Coin'] = df2['Coin'].replace(coin,'Stable Coins')

In [ ]:
# EDIT FTX_CEFI_TACTICAL_STRAT1
cond = df2['Account'].isin(["FTX - CEFI_TACTICAL_STRAT1_FTX"])
df2.loc[cond, "APY"] = 0

In [ ]:
# Add risk premium
df2 = df2.merge(risk_premium, on='Account', how='left')
df2['USD Value * Risk Premium'] = df2['Risk Premium'] * df2['USD Value']
df2

In [ ]:
df2[df2['Account']=='FTX Hedge']

In [ ]:
# process the collateral table and insert it into the liquidity tier summary
# for COFA value of collateral, use default first, there is a "Collateral" column in COFA which will override the default
cofa_collateral_p1 = cofa_original[cofa_original["Collateral"].notnull()]
cofa_collateral_p2 = cofa_original[cofa_original["Collateral"].isnull()]
cofa_collateral_p1["Default"] = cofa_collateral_p1["Collateral"]
cofa_collateral = pd.concat([cofa_collateral_p1, cofa_collateral_p2])[["Coin", "Default"]]
cofa_collateral.columns = ["Coin", "COFA"]
# Carry process
carry_collateral_p1 = carry[carry["Collateral"].notnull()]
carry_collateral_p2 = carry[carry["Collateral"].isnull()]
carry_collateral_p1["Default"] = carry_collateral_p1["Collateral"]
carry_collateral = pd.concat([carry_collateral_p1, carry_collateral_p2])[["Coin", "Default"]]
carry_collateral.columns = ["Coin", "CARRY"]

# collateral tab
collateral = collateral.merge(cofa_collateral, on = "Coin", how = "left")
collateral = collateral.merge(carry_collateral, on = "Coin", how = "left")
collateral = collateral.merge(coin_price, on = "Coin", how = "left")
collateral["User Collateral USD Value"] = collateral["User Collateral"] * collateral["Price"]
collateral["Inst Collateral USD Value"] = collateral["Inst Collateral"] * collateral["Price"]


In [ ]:
cond = (df2['USD Value']>10000000) & ((df2['APY'].isna())|(df2['APY']==0)) & (~df2['Category'].isin(['CEL Treasury','Posted Collateral','Mining','CEL Users','Investments','undeployed','underdeployed','Underdeployed', 'operational']))
df2[cond][['Coin','Category','Account','APY']]

In [ ]:
# get the whole apy-template table and insert it into the liquidity tier summary
apy_template = template.drop([0])
apy_template.fillna(value = "N/A", inplace = True)
apy_template.replace([" ", "", "  "], "N/A", inplace = True)
apy_template.columns = apy_template.columns.fillna('')

path = "excel_outputs/Liquidity_Tier_Summary Live.xlsx"
lp = openpyxl.load_workbook(path)
lp.remove(lp['Data'])
lp.remove(lp['Price'])
lp.remove(lp['Pivot_Data'])
lp.remove(lp['APY'])
lp.remove(lp['COFA'])
lp.remove(lp['Collateral'])
writer = pd.ExcelWriter(path, engine = 'openpyxl')
writer.book = lp
coin_tiers[filter3].to_excel(writer, sheet_name = 'Data', index = False)
coin_price.to_excel(writer, sheet_name = 'Price', index = False)
df2[col_names].to_excel(writer, sheet_name = 'Pivot_Data', index = False)
apy_template.to_excel(writer, sheet_name = 'APY', index = False)
cofa_original.to_excel(writer, sheet_name = 'COFA', index = False)
collateral.to_excel(writer, sheet_name = 'Collateral', index = False)
writer.close()
lp.close()

## following code will push the data to the shared google sheet

In [ ]:
assert False

In [ ]:
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import datetime as dt

In [ ]:
scope = [
'https://www.googleapis.com/auth/spreadsheets',
'https://www.googleapis.com/auth/drive',
]
last_updated = "Updated at - " + dt.datetime.utcnow().strftime("%m/%d/%Y, %H:%M:%S") +'UTC'
# this is ID for testing
#SPREADSHEET_ID = '1IptNC0hEhwvuyfI4m2kP9-rR-3jDAOhgQaPNkW2I_QQ'
# this is ID for waterfall sheet
SPREADSHEET_ID = "1ZkSLZH2QwHnfdSpQUWAv2qum6xzhemngjWQBJBn_KeM" #Real Last
#SPREADSHEET_ID = "1lw17fQ-oRTXPBVSDkiMGfWemtXGwzruCgsHjYBxlYoU" # LAST Test
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('write_token.json'):
        creds = Credentials.from_authorized_user_file('write_token.json', scope)
'''flow = InstalledAppFlow.from_client_secrets_file('client_secret_992941975507-pvjneopi7dhmj2mqq6arpghs6r7q31jj.json', scope)
creds = flow.run_local_server(port=0)
with open('write_token.json', 'w') as token:
    token.write(creds.to_json())'''
service = build('sheets', 'v4', credentials=creds)

In [ ]:
response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="Categories!A1",
    body=dict(
        majorDimension='ROWS',
        values=[[last_updated]]
)
).execute()

In [ ]:
response = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID,
    range="Data",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="Data!A1",
    body=dict(
        majorDimension='ROWS',
        values=coin_tiers[filter3].fillna(value = "N/A").T.reset_index().T.values.tolist())
).execute()

In [ ]:
response = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID,
    range="Price",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="Price!A1",
    body=dict(
        majorDimension='ROWS',
        values=coin_price.fillna(value = 0).T.reset_index().T.values.tolist())
).execute()

In [ ]:
response = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID,
    range="APY",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="APY!A1",
    body=dict(
        majorDimension='ROWS',
        values=apy_template.fillna(value = 0).T.reset_index().T.values.tolist())
).execute()

In [ ]:
response = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID,
    range="COFA",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="COFA!A1",
    body=dict(
        majorDimension='ROWS',
        values=cofa_original.fillna(value = "N/A").T.reset_index().T.values.tolist())
).execute()

In [ ]:
response = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID,
    range="CARRY",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="CARRY!A1",
    body=dict(
        majorDimension='ROWS',
        values=carry.fillna(value = "N/A").T.reset_index().T.values.tolist())
).execute()

In [ ]:
response = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID,
    range="Collateral",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="Collateral!A1",
    body=dict(
        majorDimension='ROWS',
        values=collateral.fillna(value = "N/A").T.reset_index().T.values.tolist())
).execute()

In [ ]:
response = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID,
    range="Coin_Total_Asset_Liability",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="Coin_Total_Asset_Liability!A1",
    body=dict(
        majorDimension='ROWS',
        values=coin_asset_liability.fillna(value = 0).T.reset_index().T.values.tolist())
).execute()

In [ ]:
response = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID,
    range="Pivot_Data",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="Pivot_Data!A1",
    body=dict(
        majorDimension='ROWS',
        values=df2[col_names].fillna(value = "N/A").T.reset_index().T.values.tolist())
).execute()

In [ ]:
loan_book4

In [ ]:
response = service.spreadsheets().values().clear(
    spreadsheetId=SPREADSHEET_ID,
    range="Instilend_Active_Loans_Given",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID,
    valueInputOption='RAW',
    range="Instilend_Active_Loans_Given!A1",
    body=dict(
        majorDimension='ROWS',
        values=loan_book4.fillna(value = "N/A").T.reset_index().T.values.tolist())
).execute()

## below code is for storing historical data with a date label 

# get update date and time
now = datetime.now()
dt_string = now.strftime("%Y-%m-%d %H:%M:%S")
print("Updated at:", dt_string)
date_string = dt_string[0:10]
date_string

coin_tiers["Date"] = date_string
df2["Date"] = date_string
df2_col_names = ["Date", "Coin", "Category", "Account", "Tier", "# of Coins", 
             "APY", "COFA", "Price", "USD Value", "USD Value * COFA",
             "USD Value * APY"]
coin_tiers_col_names = ['Date', 'Coin', 'Coin_Tier_1', 'Coin_Tier_2', 'Coin_Tier_3', 'Coin_Tier_4',
       'Coin_Tier_5', 'APY_Tier_1', 'APY_Tier_2', 'APY_Tier_3', 'APY_Tier_4',
       'APY_Tier_5']

# Read the liquidity data excel file and apppend new data to it
#path = "/home/fan7893/Documents/Celsius/Liquidity/Liquidity_Tier_Data.xlsx"
#path = "excel_input/Liquidity_Tier_Summary Live.xlsx"
data = coin_tiers[filter3].fillna(value = "N/A")
pivot_data = df2[col_names].fillna(value = "N/A")

# there may be some null values due to excel operation, need to filter them out
data = data[data["Date"].notnull()]
#pivot_data = pivot_data[pivot_data["Date"].notnull()]

# if running for multiple times for the same date, we need to replace the previous run data from same date
data = data[data["Date"] != date_string]
#pivot_data = pivot_data[pivot_data["Date"] != date_string]

data = pd.concat([data, coin_tiers[filter3][coin_tiers_col_names]])
pivot_data = pd.concat([pivot_data, df2[df2_col_names]])

# reinsert the updated data into the template
writer = pd.ExcelWriter(path)

# add format info
workbook = writer.book
format1 = workbook.add_format({'num_format': '$#,##0', "align": "left"})
format2 = workbook.add_format({'num_format': '#,##0.00%', "align": "left"})
format3 = workbook.add_format({'num_format': '#,##0.00', "align": "left"})
format4 = workbook.add_format({'num_format': '$#,##0.00', "align": "left"})
data.to_excel(writer, sheet_name = 'Data', index = False)
pivot_data.to_excel(writer, sheet_name = 'Pivot_Data', index = False)

# set format
worksheet = writer.sheets["Data"]
worksheet.set_column('A:B', 16)
worksheet.set_column('C:G', 18, format3)
worksheet.set_column('H:L', 18, format2)
worksheet = writer.sheets["Pivot_Data"]
worksheet.set_column('A:C', 18)
worksheet.set_column('D:D', 32)
worksheet.set_column('E:E', 16)
worksheet.set_column('F:F', 18, format3)
worksheet.set_column('G:H', 16, format2)
worksheet.set_column('I:I', 18, format4)
worksheet.set_column('J:L', 18, format1)
writer.close()

historical_data_id = "1pzlQEQE4FI4OnGrhKMfu1Skwrvs-FubYuEn-Ez5HpsM"

response = service.spreadsheets().values().clear(
    spreadsheetId=historical_data_id,
    range="Data",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=historical_data_id,
    valueInputOption='RAW',
    range="Data!A1",
    body=dict(
        majorDimension='ROWS',
        values=data.fillna(value = 0).T.reset_index().T.values.tolist())
).execute()

response = service.spreadsheets().values().clear(
    spreadsheetId=historical_data_id,
    range="Pivot_Data",
    ).execute()

response = service.spreadsheets().values().update(
    spreadsheetId=historical_data_id,
    valueInputOption='RAW',
    range="Pivot_Data!A1",
    body=dict(
        majorDimension='ROWS',
        values=pivot_data.fillna(value = "N/A").T.reset_index().T.values.tolist())
).execute()

## below code is to copy this freeze waterfall into the archive

In [ ]:
if isFreeze:
    scope = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive',
    'https://www.googleapis.com/auth/drive.file'
    ]

    # this is ID for waterfall live, but we just updated it using freeze data
    SPREADSHEET_ID = "1ZkSLZH2QwHnfdSpQUWAv2qum6xzhemngjWQBJBn_KeM"
    #SPREADSHEET_ID = "1ULOqqkYP7DDhVkZBoEA3i-Ka4x_YBPtAPuIzb84Ekrg" # GAB LAST TEST
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('drive_file_token.json'):
            creds = Credentials.from_authorized_user_file('drive_file_token.json', scope)
    '''flow = InstalledAppFlow.from_client_secrets_file('client_secret_510599516312-aj6d72c90n3fmrbf3ou6gromil06pr8c.apps.googleusercontent.com.json', scope)
    creds = flow.run_local_server(port=0)
    with open('drive_file_token.json', 'w') as token:
        token.write(creds.to_json())'''
    service = build('drive', 'v3', credentials=creds)

    # extract the most recent freeze id from the url provided and get the file name using that id
    freeze = pd.read_excel("excel_input/freeze_address.xlsx")
    freeze.columns = freeze.iloc[0]
    freeze.drop([0], inplace = True)
    freeze_address = str(freeze.iloc[-1]["Freeze URL"])
    #print(freeze_address)
    freeze_id = freeze_address.split("/")[5]

    #get the file name of most recent freeze sheet, which include the date of the freeze
    response = service.files().get(fileId=freeze_id).execute()
    freeze_name  ="Portfolio Waterfall TEST - " + response["name"]

    # after updating the portfolio waterall live, copied it to the archive folder
    archive_folder_id = "13f2xspl16wzdHBRwhgnEkKj80zLL9Rzu"
    newfile = {'name': freeze_name,  'parents' : [archive_folder_id]}
    response = service.files().copy(fileId=SPREADSHEET_ID, body=newfile).execute()

    # get link
    link_base = "https://docs.google.com/spreadsheets/d/"
    link = link_base + response['id']
    date = freeze_last_date

In [ ]:
scope = [
'https://www.googleapis.com/auth/spreadsheets',
'https://www.googleapis.com/auth/drive',
]
# this is ID for testing
SPREADSHEET_ID = "1dBf2U0nfHzDqtkrfxdm8P2hfwM_Ndb9nx4OW8HCwmcg"
if os.path.exists('write_token.json'):
        creds = Credentials.from_authorized_user_file('write_token.json', scope)
service = build('sheets', 'v4', credentials=creds)

response = service.spreadsheets().values().append(
    spreadsheetId = SPREADSHEET_ID,
    range = "Past Waterfall Freezes!A2:B4",
    body = {
        "majorDimension": "ROWS",
        "values": [[freeze_name_date,link]]
    },
    valueInputOption = 'RAW'
).execute()


"""
response = service.spreadsheets().values().batchUpdate(
    spreadsheetId=SPREADSHEET_ID,
    body={
        "valueInputOption":'RAW',
        'data':{
            'range':
            [[1,2]],
        'ValueRanges': [[date,link]]
}}).execute()"""

# Save in history

# Connect to Drive API
spreadsheet_id = '1pzlQEQE4FI4OnGrhKMfu1Skwrvs-FubYuEn-Ez5HpsM'
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']
creds = None
creds = Credentials.from_authorized_user_file('token.json', SCOPES)
service = build('sheets', 'v4', credentials=creds)

# Call the Drive v3 API
SAMPLE_RANGE_NAME = 'Pivot_Data!A3:T'
request = service.spreadsheets().values().batchGet(
    spreadsheetId=spreadsheet_id,
    ranges=SAMPLE_RANGE_NAME,
    valueRenderOption='FORMATTED_VALUE')
response = request.execute()

timestamp = datetime.now().strftime("%m/%d/%Y %H:%M:%S")
today = datetime.now().strftime("%m/%d/%Y")

history = pd.DataFrame(response['valueRanges'][0]['values'])
history.columns = col_names + ['Timestamp']
history['Timestamp'] = pd.to_datetime(history['Timestamp'])
cond = history['Timestamp'].dt.strftime("%m/%d/%Y") == today
idx_todrop= history[cond].index
history.drop(idx_todrop, inplace=True)

history

pivot_data = df2[col_names].fillna(value = "N/A")
pivot_data['Timestamp'] = timestamp
pivot_data = pd.concat([history, pivot_data])
id = '1pzlQEQE4FI4OnGrhKMfu1Skwrvs-FubYuEn-Ez5HpsM'

pivot_data

scope = [
'https://www.googleapis.com/auth/spreadsheets',
'https://www.googleapis.com/auth/drive',
]
# this is ID for testing
#SPREADSHEET_ID = '1IptNC0hEhwvuyfI4m2kP9-rR-3jDAOhgQaPNkW2I_QQ'
# this is ID for waterfall sheet
SPREADSHEET_ID = '1pzlQEQE4FI4OnGrhKMfu1Skwrvs-FubYuEn-Ez5HpsM' #Historical Data
range = 'Pivot_Data!A:T2'
if os.path.exists('write_token.json'):
        creds = Credentials.from_authorized_user_file('write_token.json', scope)
service = build('sheets', 'v4', credentials=creds)

data = pivot_data.values.tolist()

response = service.spreadsheets().values().append(
    spreadsheetId = SPREADSHEET_ID,
    range = range,
    body = {
        "majorDimension": "ROWS",
        "values": data
    },
    valueInputOption = 'RAW'
).execute()

freeze_name